In [1]:
import torch
# Import sentence transformer package. More information can be found here: https://www.sbert.net/
from sentence_transformers import SentenceTransformer, util

In [13]:
# Preprocessing the test data code. Concatenate all the gold standard results into one file.
# So long as the input and gold standard files are in the same order then this is safe.
import glob
import shutil

indexes = []
index = 0

# Concatenate all the results into one file
outFileName = "data/Test_Data/Result_Files/mergedGoldStandard.txt"
with open(outFileName, 'w', encoding='utf8') as outfile:
    for filename in glob.glob('data/Test_Data/Gold_Standard_Files/*.txt'):
        if filename == outFileName:
            # don't want to copy the output into the output
            continue
        with open(filename, 'r', encoding='utf8') as readfile:
            for line in readfile:
                # Remove all the empty lines
                if not line.isspace():
                    indexes.append(index)
                    outfile.write(line)
                index += 1
print(indexes)

index = 0
list_index = 0
# Merge all the input files into one file
outFileName = "data/Test_Data/Result_Files/mergedInputFiles.txt"
with open(outFileName, 'w', encoding='utf8') as outfile:
    for filename in glob.glob('data/Test_Data/Input_Files/*.txt'):
        if filename == outFileName:
            # don't want to copy the output into the output
            continue
        with open(filename, 'r', encoding='utf8') as readfile:
            for line in readfile:
                if list_index >= len(indexes):
                    break
                if index == indexes[list_index]:
                    outfile.write(line)
                    list_index += 1
                index += 1

print(index)
# Note that we need to modify the gold standard files slightly so that the values are compressed to a range 0-1.
# As they are currently on a scale from 0-5 we need only divide them by 5. Rounded to 5 decimal places.

# Concatenate all the results into one file
outFileName = "data/Training_Data/Result_Files/mergedGoldStandard.txt"
with open(outFileName, 'w', encoding='utf8') as outfile:
    for filename in glob.glob('data/Training_Data/Gold_Standard_Files/*.txt'):
        with open(filename, 'r', encoding='utf8') as readfile:
            for line in readfile:
                # Remove all the empty lines
                if not line.isspace():
                    # Convert the string to a double and then round it to the nearest integer
                    val = str(round((float(line) / 5), 5)) + "\n"
                    outfile.write(val)

# Merge all the input files into one file
outFileName = "data/Training_Data/Result_Files/mergedInputFiles.txt"
with open(outFileName, 'w', encoding='utf8') as outfile:
    for filename in glob.glob('data/Test_Data/Input_Files/*.txt'):
        with open(filename, 'r', encoding='utf8') as readfile:
            shutil.copyfileobj(readfile, outfile)

print('Done pre-processing training and test data.')

[8, 15, 28, 34, 40, 41, 47, 48, 61, 65, 80, 87, 92, 102, 107, 110, 116, 120, 121, 132, 136, 139, 141, 142, 155, 159, 175, 184, 189, 193, 202, 209, 212, 215, 227, 232, 236, 239, 242, 249, 254, 258, 271, 272, 286, 291, 292, 298, 305, 313, 314, 315, 318, 322, 325, 334, 337, 352, 356, 359, 367, 370, 372, 376, 385, 389, 391, 396, 400, 410, 411, 415, 428, 429, 432, 433, 436, 445, 447, 450, 470, 471, 476, 490, 494, 496, 502, 516, 521, 535, 537, 548, 549, 567, 571, 581, 582, 583, 599, 603, 607, 615, 616, 628, 629, 639, 641, 643, 648, 650, 659, 660, 662, 664, 666, 675, 678, 682, 683, 684, 694, 695, 704, 705, 713, 718, 721, 723, 736, 737, 742, 743, 745, 749, 750, 761, 768, 770, 778, 779, 784, 787, 806, 829, 831, 854, 855, 861, 862, 868, 869, 871, 875, 877, 879, 882, 887, 889, 892, 901, 908, 909, 916, 921, 925, 937, 945, 947, 950, 955, 961, 965, 966, 975, 976, 987, 990, 998, 1012, 1013, 1014, 1046, 1052, 1062, 1063, 1065, 1066, 1072, 1077, 1080, 1088, 1099, 1106, 1107, 1111, 1127, 1135, 1150, 115

In [15]:
gold_standard = []
readFileName = "data/Test_Data/Result_Files/mergedGoldStandard.txt"
with open(readFileName, 'r', encoding='utf8') as readFileName:
    for line in readFileName.readlines():
        gold_standard.append(int(line))
print(len(gold_standard))

# If you have a cuda capable device we will send the tensors to that
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

# Read in the test data into vectors.
readFileName = "data/Test_data/Result_Files/mergedInputFiles.txt"
sentences1, sentences2 = [], []
with open(readFileName, 'r', encoding='utf8') as readFileName:
    for line in readFileName.readlines():
        sentences = line.split('\t')
        sentences1.append(sentences[0])
        sentences2.append(sentences[1])

print(len(sentences1))

1186
cuda:0
1186


In [4]:
# Reading the training data into vectors.
from sentence_transformers import SentencesDataset, InputExample, losses

training_sentences_1, training_sentences_2, training_labels = [], [], []

# Now we read in the sentences and their labels
# Read in the input files
readFileName = "data/Training_Data/Result_Files/mergedInputFiles.txt"
with open(readFileName, 'r', encoding='utf8') as readFileName:
    for line in readFileName.readlines():
        sentences = line.split('\t')
        training_sentences_1.append(sentences[0])
        training_sentences_2.append(sentences[1])

readFileName = "data/Training_Data/Result_Files/mergedGoldStandard.txt"
with open(readFileName, 'r', encoding='utf8') as readFileName:
    for line in readFileName.readlines():
        training_labels.append(float(line))

train_examples = []
for sent_1, sent_2, label in zip(training_sentences_1, training_sentences_2, training_labels):
    train_examples.append(InputExample(texts=[sent_1, sent_2], label=label))

In [5]:
# In this module we, will train one of the transformer models and leave the other in its pre-trained state.
# We train this model using the following steps: https://www.sbert.net/examples/training/sts/README.html, https://www.sbert.net/docs/training/overview.html
from torch.utils.data import DataLoader

# The following are SBERT models. More information here: https://www.sbert.net/docs/package_reference/models.html#main-classes
model_name = 'distiluse-base-multilingual-cased-v2'
# Place the models into an array so we can iterate over them
models = {'With_Training': SentenceTransformer(model_name), 'Without_Training': SentenceTransformer(model_name)}


# Train the model
training_model = models['With_Training']

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=5)
train_loss = losses.CosineSimilarityLoss(training_model)

training_model.fit(train_objectives=[(train_dataloader, train_loss)], show_progress_bar=True)

In [6]:
# Run each model on the test data and write their output to a file
for key, model in models.items():
    resultsFileName = f'data/Result_Files/{key}_{model_name}.txt'
    #Compute embedding for both lists
    embeddings1 = model.encode(sentences1, convert_to_tensor=True, device=device)
    embeddings2 = model.encode(sentences2, convert_to_tensor=True, device=device)

    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)

    # TODO: Confirm taking the absolute value is correct here
    # Normalize cosine_scores by taking the absolute value and multiplying by 5 then rounding. Note, the multiplication broadcasts over the tensor.
    cosine_scores = torch.round(abs(cosine_scores) * 5)

    with open(resultsFileName, 'w', encoding='utf8') as outfile:
        for i in range(len(sentences1)):
            outfile.writelines(str(int(cosine_scores[i][i].item())) + '\n')

print('Done testing phase.')

RuntimeError: CUDA out of memory. Tried to allocate 1.26 GiB (GPU 0; 4.00 GiB total capacity; 2.34 GiB already allocated; 0 bytes free; 2.70 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Extra code for comparing the pre-trained model vs the additionally trained model

gold_standard, with_training, without_training = [], [], []

readFileName = "data/Compare/mergedGoldStandard.txt"
with open(readFileName, 'r', encoding='utf8') as readFileName:
    for line in readFileName.readlines():
        gold_standard.append(int())